In [1]:
import pandas as pd
import seaborn as sns

In [66]:
df1=pd.read_csv("Earnings_Hours_Worked_Ireland.csv")

In [67]:
df1.head()

,C01903V02345,Type of Worker,TLIST(A1),Year,STATISTIC,Statistic Label,UNIT,VALUE
0,2,All manual employee grades,1969,1969,BDA01C1,Average Earnings per Hour in the Building and ...,Euro,NaN
1,2,All manual employee grades,1969,1969,BDA01C2,Average Earnings per Week in the Building and ...,Euro,NaN
2,2,All manual employee grades,1969,1969,BDA01C3,Average Hours Worked per Week in the Building ...,Number,NaN
3,2,All manual employee grades,1970,1970,BDA01C1,Average Earnings per Hour in the Building and ...,Euro,NaN
4,2,All manual employee grades,1970,1970,BDA01C2,Average Earnings per Week in the Building and ...,Euro,NaN


In [68]:
df1=df1.drop(["C01903V02345", "TLIST(A1)", "STATISTIC"], axis=1)

In [69]:
df1.head(60)

,Type of Worker,Year,Statistic Label,UNIT,VALUE
0,All manual employee grades,1969,Average Earnings per Hour in the Building and ...,Euro,NaN
1,All manual employee grades,1969,Average Earnings per Week in the Building and ...,Euro,NaN
2,All manual employee grades,1969,Average Hours Worked per Week in the Building ...,Number,NaN
3,All manual employee grades,1970,Average Earnings per Hour in the Building and ...,Euro,NaN
4,All manual employee grades,1970,Average Earnings per Week in the Building and ...,Euro,NaN
5,All manual employee grades,1970,Average Hours Worked per Week in the Building ...,Number,NaN
6,All manual employee grades,1971,Average Earnings per Hour in the Building and ...,Euro,NaN
7,All manual employee grades,1971,Average Earnings per Week in the Building and ...,Euro,NaN
8,All manual employee grades,1971,Average Hours Worked per Week in the Building ...,Number,NaN
9,All manual employee grades,1972,Average Earnings per Hour in the Building and ...,Euro,NaN


In [70]:
df1.tail()

,Type of Worker,Year,Statistic Label,UNIT,VALUE
1075,All employees (clerical and operatives),2007,Average Earnings per Week in the Building and ...,Euro,811.38
1076,All employees (clerical and operatives),2007,Average Hours Worked per Week in the Building ...,Number,44.00
1077,All employees (clerical and operatives),2008,Average Earnings per Hour in the Building and ...,Euro,19.03
1078,All employees (clerical and operatives),2008,Average Earnings per Week in the Building and ...,Euro,821.50
1079,All employees (clerical and operatives),2008,Average Hours Worked per Week in the Building ...,Number,43.20


In [73]:
df1 = df1.drop(index=df1.index[:51]).reset_index(drop=True)

In [74]:
df1.head()

,Type of Worker,Year,Statistic Label,UNIT,VALUE
0,All manual employee grades,2002,Average Hours Worked per Week in the Building ...,Number,45.00
1,All manual employee grades,2003,Average Earnings per Hour in the Building and ...,Euro,15.56
2,All manual employee grades,2003,Average Earnings per Week in the Building and ...,Euro,693.71
3,All manual employee grades,2003,Average Hours Worked per Week in the Building ...,Number,44.60
4,All manual employee grades,2004,Average Earnings per Hour in the Building and ...,Euro,16.24


In [75]:
df1.dtypes

Type of Worker      object
Year                 int64
Statistic Label     object
UNIT                object
VALUE              float64
dtype: object

In [76]:
df1null = df1.isnull().sum()

In [77]:
df1null

Type of Worker       0
Year                 0
Statistic Label      0
UNIT                 0
VALUE              306
dtype: int64

In [79]:
df1.dropna(inplace=True)

In [80]:
df1.describe

<bound method NDFrame.describe of                               Type of Worker  Year  \
0                 All manual employee grades  2002   
1                 All manual employee grades  2003   
2                 All manual employee grades  2003   
3                 All manual employee grades  2003   
4                 All manual employee grades  2004   
..                                       ...   ...   
974  All employees (clerical and operatives)  2007   
975  All employees (clerical and operatives)  2007   
976  All employees (clerical and operatives)  2008   
977  All employees (clerical and operatives)  2008   
978  All employees (clerical and operatives)  2008   

                                       Statistic Label    UNIT   VALUE  
0    Average Hours Worked per Week in the Building ...  Number   45.00  
1    Average Earnings per Hour in the Building and ...    Euro   15.56  
2    Average Earnings per Week in the Building and ...    Euro  693.71  
3    Average Hours Worked

In [4]:
df2=pd.read_csv("Production_Europe.tsv",sep="\t")
#revers to the tab - file is .tsv#

In [5]:
df2.head()

,"indic_bt,nace_r2,s_adj,unit,geo\time",2022,2021,2020,2019,2018,2017,2016,2015,2014,...,1989,1988,1987,1986,1985,1984,1983,1982,1981,1980
0,"PROD,B-D_F,CA,I15,AT",128.4,120.7,109.3,115.7,114.2,108.4,102.2,100.0,98.7,...,:,:,:,:,:,:,:,:,:,:
1,"PROD,B-D_F,CA,I15,BE",121.0 s,121.4 s,106.2 s,111.4 s,107.4 s,106.1 s,103.7 s,100.0 s,101.5 s,...,:,:,:,:,:,:,:,:,:,:
2,"PROD,B-D_F,CA,I15,BG",119.5 s,107.5 s,98.6 s,105.1 s,104.0 s,103.5 s,99.3 s,100.0 s,96.1 s,...,:,:,:,:,:,:,:,:,:,:
3,"PROD,B-D_F,CA,I15,CH",124.6,119.2,110.5,114.2,110.4,105.3,100.1,100.0,102.1,...,:,:,:,:,:,:,:,:,:,:
4,"PROD,B-D_F,CA,I15,CY",148.6 s,150.5 s,142.3 s,153.1 s,142.9 s,128.9 s,112.5 s,100.0 s,96.7 s,...,:,:,:,:,:,:,:,:,:,:


In [122]:
df2_transposed = df2.set_index('indic_bt,nace_r2,s_adj,unit,geo\\time').T

In [123]:
df2.head()

,"indic_bt,nace_r2,s_adj,unit,geo\time",2022,2021,2020,2019,2018,2017,2016,2015,2014,...,1984,1983,1982,1981,1980,indic_bt,nace_r2,s_adj,unit,geo
0,"PROD,B-D_F,CA,I15,AT",128.4,120.7,109.3,115.7,114.2,108.4,102.2,100.0,98.7,...,:,:,:,:,:,NaN,NaN,NaN,NaN,NaN
1,"PROD,B-D_F,CA,I15,BE",121.0 s,121.4 s,106.2 s,111.4 s,107.4 s,106.1 s,103.7 s,100.0 s,101.5 s,...,:,:,:,:,:,NaN,NaN,NaN,NaN,NaN
2,"PROD,B-D_F,CA,I15,BG",119.5 s,107.5 s,98.6 s,105.1 s,104.0 s,103.5 s,99.3 s,100.0 s,96.1 s,...,:,:,:,:,:,NaN,NaN,NaN,NaN,NaN
3,"PROD,B-D_F,CA,I15,CH",124.6,119.2,110.5,114.2,110.4,105.3,100.1,100.0,102.1,...,:,:,:,:,:,NaN,NaN,NaN,NaN,NaN
4,"PROD,B-D_F,CA,I15,CY",148.6 s,150.5 s,142.3 s,153.1 s,142.9 s,128.9 s,112.5 s,100.0 s,96.7 s,...,:,:,:,:,:,NaN,NaN,NaN,NaN,NaN


In [94]:
df3=pd.read_csv("national_house_construction_cost_index_0.csv")

In [95]:
df3.head()

,Unnamed: 0,NATIONAL HOUSE CONSTRUCTION COST INDEX,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,Base: January 1991 = 100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Month,1994,1995,1996,1997,1998,1999,2000,2001,2002,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016.0
2,January,109.2,113.5,115.9,118,122.9,126.3,135.8,154.3,169.9,...,198.8,207.4,208,206.9,210.2,202.5,204.1,204.9,206.2,207.4
3,February,109.3,114.1,115.7,118.5,123.1,126.4,136.1,154.7,170,...,198.8,208.2,207.3,207.2,210.5,202.6,204.2,205.4,206,207.1
4,March,109.4,114.1,115.9,118.8,123.8,126.5,136.4,155.6,169.5,...,199.4,208.6,206.8,207.3,200.7,202.7,204.3,205.4,206.5,NaN


In [96]:
df3=pd.read_csv("national_house_construction_cost_index_0.csv", header=2)

In [97]:
df3.head()

,Month,1994,1995,1996,1997,1998,1999,2000,2001,2002,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,January,109.2,113.5,115.9,118,122.9,126.3,135.8,154.3,169.9,...,198.8,207.4,208,206.9,210.2,202.5,204.1,204.9,206.2,207.4
1,February,109.3,114.1,115.7,118.5,123.1,126.4,136.1,154.7,170,...,198.8,208.2,207.3,207.2,210.5,202.6,204.2,205.4,206,207.1
2,March,109.4,114.1,115.9,118.8,123.8,126.5,136.4,155.6,169.5,...,199.4,208.6,206.8,207.3,200.7,202.7,204.3,205.4,206.5,NaN
3,April,110.3,114.1,115.9,119.8,123.8,130.5,136.5,157.4,169.5,...,200,208.9,206.5,207.9,201,203.4,204.5,205.6,206.8,NaN
4,May,110.1,114.1,115.9,119.8,124.5,130.5,136.8,157.6,169.5,...,200.1,209.4,206.5,208.7,201.2,203.6,204.5,205.9,207.2,NaN


In [98]:
df3.dtypes

Month     object
1994      object
1995      object
1996      object
1997      object
1998      object
1999      object
2000      object
2001      object
2002      object
2003      object
2004      object
2005      object
2006      object
2007      object
2008      object
2009      object
2010      object
2011      object
2012      object
2013      object
2014      object
2015      object
2016     float64
dtype: object

In [99]:
df3_transposed = df3.set_index('Month').T

In [101]:
df3_transposed.head()

Month,January,February,March,April,May,June,July,August,September,October,November,December,Yearly average,% Increase on previous year
1994,109.2,109.3,109.4,110.3,110.1,110.6,110.7,110.9,112.2,112.7,112.9,113.1,111,3.40%
1995,113.5,114.1,114.1,114.1,114.1,114.1,114.5,114.5,115.7,115.8,115.9,115.9,114.7,3.30%
1996,115.9,115.7,115.9,115.9,115.9,116,116,116,116.9,117.1,117.2,117.2,116.3,1.40%
1997,118,118.5,118.8,119.8,119.8,119.9,120.8,121,121.5,122,122.6,122.6,120.4,3.50%
1998,122.9,123.1,123.8,123.8,124.5,124.5,125.7,125.7,126,126.2,126.2,126.1,124.9,3.70%


In [102]:
df5=pd.read_csv("download1061915290288055378.csv")

In [103]:
df5.head()

,FeatureCode,DateCode,Measurement,Units,Value,Construction indices
0,K02000001,2011 Feb,Costs,NaN,103.1,New Housing
1,K02000001,2011 Feb,Costs,NaN,104.4,All work
2,K02000001,2011 Feb,Costs,NaN,105.0,Other new work
3,K02000001,2011 Feb,Costs,NaN,103.7,Repairs and maintenance
4,K02000001,2016 Aug,Costs,NaN,110.1,New Housing


In [104]:
df5.tail()

,FeatureCode,DateCode,Measurement,Units,Value,Construction indices
488,K02000001,2018 Apr,Costs,NaN,119.1,Repairs and maintenance
489,K02000001,2018 Apr,Costs,NaN,118.4,All work
490,K02000001,2012 Sep,Costs,NaN,109.8,All work
491,K02000001,2002 Q1,Costs,NaN,117.1,Index of hourly wage costs
492,K02000001,2012 Sep,Costs,NaN,107.1,New Housing


In [105]:
df5 = df5.drop(["FeatureCode","Measurement","Units"], axis=1)

In [106]:
df5.head()

,DateCode,Value,Construction indices
0,2011 Feb,103.1,New Housing
1,2011 Feb,104.4,All work
2,2011 Feb,105.0,Other new work
3,2011 Feb,103.7,Repairs and maintenance
4,2016 Aug,110.1,New Housing


In [107]:
df5.dtypes

DateCode                 object
Value                   float64
Construction indices     object
dtype: object

In [108]:
df6=pd.read_csv("BBA02.20230428105548.csv")

In [109]:
df6.head()

,STATISTIC,Statistic Label,C02196V02652,State,TLIST(A1),Year,UNIT,VALUE
0,BBA02,Annual Average Index of Employment (1975 - Dat...,-,State,1975,1975,Base 2000=100,133.2
1,BBA02,Annual Average Index of Employment (1975 - Dat...,-,State,1976,1976,Base 2000=100,128.1
2,BBA02,Annual Average Index of Employment (1975 - Dat...,-,State,1977,1977,Base 2000=100,134.8
3,BBA02,Annual Average Index of Employment (1975 - Dat...,-,State,1978,1978,Base 2000=100,147.0
4,BBA02,Annual Average Index of Employment (1975 - Dat...,-,State,1979,1979,Base 2000=100,158.8


In [110]:
df7=pd.read_csv("Labourinput_Europe.tsv.gz", sep="\t")
#reverts to the tab - file is .tsv#

In [111]:
df7.head()

,"indic_bt,nace_r2,s_adj,unit,geo\time",2022,2021,2020,2019,2018,2017,2016,2015,2014,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
0,"EMPL,F,NSA,I10,AL",:,:,:,:,:,:,123.0,115.8,103.9,...,:,:,:,:,:,:,:,:,:,:
1,"EMPL,F,NSA,I10,AT",:,:,:,:,:,103.3 p,101.0,100.9,102.4,...,104.8,109.5,101.1,102.1,:,:,:,:,:,:
2,"EMPL,F,NSA,I10,BE",:,:,:,:,:,:,101.9 p,100.2,100.8,...,:,:,:,:,:,:,:,:,:,:
3,"EMPL,F,NSA,I10,BG",:,:,:,:,:,70.3 p,71.4 p,74.9 p,81.9,...,:,:,:,:,:,:,:,:,:,:
4,"EMPL,F,NSA,I10,CH",:,:,:,:,:,103.5,103.9,105.1,105.0,...,88.0,89.1,:,:,:,:,:,:,:,:


In [112]:
df7.tail()

,"indic_bt,nace_r2,s_adj,unit,geo\time",2022,2021,2020,2019,2018,2017,2016,2015,2014,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
478,"WAGE,F,NSA,PCH_SM,SE",:,:,-3.1 e,8.5 e,6.0 e,4.3 e,8.2 e,8.7 e,3.6 e,...,:,:,:,:,:,:,:,:,:,:
479,"WAGE,F,NSA,PCH_SM,SI",:,:,6.0,2.3,4.2,2.6,1.4,-0.5,-0.1,...,:,:,:,:,:,:,:,:,:,:
480,"WAGE,F,NSA,PCH_SM,SK",:,:,-3.6,12.7,10.8,6.0,4.7,4.5,-1.8,...,:,:,:,:,:,:,:,:,:,:
481,"WAGE,F,NSA,PCH_SM,TR",:,:,5.4,-6.2,13.2,15.0,28.3,17.9,15.8,...,:,:,:,:,:,:,:,:,:,:
482,"WAGE,F,NSA,PCH_SM,UK",:,:,:,7.6,7.7,6.4,7.4,5.6,4.4,...,:,:,:,:,:,:,:,:,:,:


In [135]:
df8=pd.read_csv("PriceIndex_BuildingConstruction_Materials_Ireland.csv")

In [136]:
df8.tail(15)

,STATISTIC,Statistic Label,TLIST(A1),Year,C01409V03262,Type of Material,UNIT,VALUE
625,WPA15C02,Percentage Change over 12 months in Wholesale ...,2022,2022,691,Electrical fittings,%,9.9
626,WPA15C02,Percentage Change over 12 months in Wholesale ...,2022,2022,69162,Lighting equipment,%,0.0
627,WPA15C02,Percentage Change over 12 months in Wholesale ...,2022,2022,69163,Protection and communication equipment,%,5.0
628,WPA15C02,Percentage Change over 12 months in Wholesale ...,2022,2022,70,All other materials,%,14.5
629,WPA15C02,Percentage Change over 12 months in Wholesale ...,2022,2022,700,Plumbing materials including sanitary ware,%,21.9
630,WPA15C02,Percentage Change over 12 months in Wholesale ...,2022,2022,701,HVAC (heating and ventilation equipment),%,5.2
631,WPA15C02,Percentage Change over 12 months in Wholesale ...,2022,2022,702,Insulating materials,%,15.6
632,WPA15C02,Percentage Change over 12 months in Wholesale ...,2022,2022,703,Pipes and fittings,%,19.1
633,WPA15C02,Percentage Change over 12 months in Wholesale ...,2022,2022,70371,PVC pipes and fittings,%,25.3
634,WPA15C02,Percentage Change over 12 months in Wholesale ...,2022,2022,70372,Copper pipes and fittings,%,16.9


In [137]:
df8=df8.drop(["STATISTIC", "TLIST(A1)"], axis=1)

In [138]:
df8.head()

,Statistic Label,Year,C01409V03262,Type of Material,UNIT,VALUE
0,Wholesale Price Index (Excl VAT) for Building ...,2015,-,Materials,Base 2015=100,100.0
1,Wholesale Price Index (Excl VAT) for Building ...,2015,601,"Stone, sand and gravel",Base 2015=100,100.0
2,Wholesale Price Index (Excl VAT) for Building ...,2015,60161,Sand and gravel,Base 2015=100,100.0
3,Wholesale Price Index (Excl VAT) for Building ...,2015,60162,Stone,Base 2015=100,100.0
4,Wholesale Price Index (Excl VAT) for Building ...,2015,611,Cement,Base 2015=100,100.0


In [157]:
df9=pd.read_csv("Purchases_BuildingConstruction_New Basis_Ireland.csv")

In [158]:
df9.head()

,STATISTIC,Statistic Label,C02407V02905,Building Sector,C01989V02424,Type of Purchases,TLIST(A1),Year,UNIT,VALUE
0,BAA07,Purchases in Building and Construction Industr...,45,All construction (45),23,"All purchases of materials, fuel and energy",2005,2005,Euro Thousand,3641555.0
1,BAA07,Purchases in Building and Construction Industr...,45,All construction (45),23,"All purchases of materials, fuel and energy",2006,2006,Euro Thousand,4371160.0
2,BAA07,Purchases in Building and Construction Industr...,45,All construction (45),23,"All purchases of materials, fuel and energy",2007,2007,Euro Thousand,4009412.0
3,BAA07,Purchases in Building and Construction Industr...,45,All construction (45),231,Materials,2005,2005,Euro Thousand,3509030.0
4,BAA07,Purchases in Building and Construction Industr...,45,All construction (45),231,Materials,2006,2006,Euro Thousand,4212084.0


In [159]:
df9.tail()

,STATISTIC,Statistic Label,C02407V02905,Building Sector,C01989V02424,Type of Purchases,TLIST(A1),Year,UNIT,VALUE
229,BAA07,Purchases in Building and Construction Industr...,455,Renting of construction or demolition equipmen...,23224,Fuel oil for stationary uses - other fuel oil,2006,2006,Euro Thousand,NaN
230,BAA07,Purchases in Building and Construction Industr...,455,Renting of construction or demolition equipmen...,23224,Fuel oil for stationary uses - other fuel oil,2007,2007,Euro Thousand,NaN
231,BAA07,Purchases in Building and Construction Industr...,455,Renting of construction or demolition equipmen...,233,Electricity,2005,2005,Euro Thousand,226.0
232,BAA07,Purchases in Building and Construction Industr...,455,Renting of construction or demolition equipmen...,233,Electricity,2006,2006,Euro Thousand,271.0
233,BAA07,Purchases in Building and Construction Industr...,455,Renting of construction or demolition equipmen...,233,Electricity,2007,2007,Euro Thousand,571.0


In [160]:
df9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   STATISTIC          234 non-null    object 
 1   Statistic Label    234 non-null    object 
 2   C02407V02905       234 non-null    int64  
 3   Building Sector    234 non-null    object 
 4   C01989V02424       234 non-null    int64  
 5   Type of Purchases  234 non-null    object 
 6   TLIST(A1)          234 non-null    int64  
 7   Year               234 non-null    int64  
 8   UNIT               234 non-null    object 
 9   VALUE              173 non-null    float64
dtypes: float64(1), int64(4), object(5)
memory usage: 18.4+ KB


In [164]:
df9.describe(include="all")

,Building Sector,Type of Purchases,Year,UNIT,VALUE
count,234,234,234.000000,234,1.730000e+02
unique,6,13,NaN,1,NaN
top,All construction (45),"All purchases of materials, fuel and energy",NaN,Euro Thousand,NaN
freq,39,18,NaN,234,NaN
mean,NaN,NaN,2006.000000,NaN,2.817845e+05
std,NaN,NaN,0.818247,NaN,8.884327e+05
min,NaN,NaN,2005.000000,NaN,0.000000e+00
25%,NaN,NaN,2005.000000,NaN,1.523000e+03
50%,NaN,NaN,2006.000000,NaN,9.888000e+03
75%,NaN,NaN,2007.000000,NaN,5.581500e+04


In [161]:
df9.astype("object").describe()

,STATISTIC,Statistic Label,C02407V02905,Building Sector,C01989V02424,Type of Purchases,TLIST(A1),Year,UNIT,VALUE
count,234,234,234,234,234,234,234,234,234,173.0
unique,1,1,6,6,13,13,3,3,1,171.0
top,BAA07,Purchases in Building and Construction Industr...,45,All construction (45),23,"All purchases of materials, fuel and energy",2005,2005,Euro Thousand,0.0
freq,234,234,39,39,18,18,78,78,234,2.0


In [162]:
df9=df9.drop(["STATISTIC", "C02407V02905", "C01989V02424", "Statistic Label", "TLIST(A1)"], axis=1)

In [163]:
df9.head()

,Building Sector,Type of Purchases,Year,UNIT,VALUE
0,All construction (45),"All purchases of materials, fuel and energy",2005,Euro Thousand,3641555.0
1,All construction (45),"All purchases of materials, fuel and energy",2006,Euro Thousand,4371160.0
2,All construction (45),"All purchases of materials, fuel and energy",2007,Euro Thousand,4009412.0
3,All construction (45),Materials,2005,Euro Thousand,3509030.0
4,All construction (45),Materials,2006,Euro Thousand,4212084.0
